In [29]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

#=====================================================================================
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        print (final_outputs)
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        print (final_outputs)
        return final_outputs
#========================================================================================
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 50
output_nodes = 10

# learning rate
learning_rate = 0.9

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

#=============================================================================================


# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 3
print ("==================== Inicializando el entrenamiento ====================")
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

print ("===================== Finalizando el entrenamiento =====================")


# load the mnist test data CSV file into a list
test_data_file = open("mnist_dataset/mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []
print ("======================== empezar procesar datos ========================")
# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    pass

print (scorecard)

==================== Inicializando el entrenamiento ====================
[[ 0.48825466]
 [ 0.57788594]
 [ 0.49747038]
 [ 0.48499247]
 [ 0.45584506]
 [ 0.6735824 ]
 [ 0.40003135]
 [ 0.4997328 ]
 [ 0.47271038]
 [ 0.35247712]]
[[ 0.09450904]
 [ 0.09807695]
 [ 0.03943638]
 [ 0.08400563]
 [ 0.06390378]
 [ 0.82824124]
 [ 0.16871189]
 [ 0.10768947]
 [ 0.0824751 ]
 [ 0.20778208]]
[[ 0.43068021]
 [ 0.12966753]
 [ 0.10319015]
 [ 0.13120611]
 [ 0.13227624]
 [ 0.38648581]
 [ 0.12810243]
 [ 0.12792057]
 [ 0.1211497 ]
 [ 0.12247686]]
[[ 0.12881202]
 [ 0.10275546]
 [ 0.07108841]
 [ 0.09613677]
 [ 0.38449035]
 [ 0.12931711]
 [ 0.101433  ]
 [ 0.08822735]
 [ 0.08523529]
 [ 0.09882389]]
[[ 0.11275756]
 [ 0.34086374]
 [ 0.05159723]
 [ 0.06797893]
 [ 0.16675944]
 [ 0.07775015]
 [ 0.07956442]
 [ 0.07043116]
 [ 0.05679415]
 [ 0.08001267]]
[[ 0.08564136]
 [ 0.10764133]
 [ 0.04233635]
 [ 0.06438973]
 [ 0.11575679]
 [ 0.0618164 ]
 [ 0.06938245]
 [ 0.06205449]
 [ 0.05661392]
 [ 0.31918096]]
[[ 0.09237416]
 [ 0.1

[[ 0.0160674 ]
 [ 0.29673903]
 [ 0.04887259]
 [ 0.04677098]
 [ 0.09892944]
 [ 0.22790553]
 [ 0.01452179]
 [ 0.03861893]
 [ 0.11498927]
 [ 0.01849747]]
[[ 0.03169696]
 [ 0.02034688]
 [ 0.05085438]
 [ 0.00799541]
 [ 0.64404605]
 [ 0.07977786]
 [ 0.0633665 ]
 [ 0.09226481]
 [ 0.03211469]
 [ 0.02881665]]
[[ 0.00639184]
 [ 0.1826749 ]
 [ 0.04528024]
 [ 0.03858108]
 [ 0.85245767]
 [ 0.12486313]
 [ 0.17947008]
 [ 0.03240791]
 [ 0.05752091]
 [ 0.09011012]]
[[ 0.03725661]
 [ 0.12662227]
 [ 0.03824518]
 [ 0.06827012]
 [ 0.29275415]
 [ 0.02926095]
 [ 0.56876947]
 [ 0.01785225]
 [ 0.0691685 ]
 [ 0.02091455]]
[[ 0.74467567]
 [ 0.04927616]
 [ 0.07492571]
 [ 0.05141209]
 [ 0.07126975]
 [ 0.05056671]
 [ 0.0670711 ]
 [ 0.09804301]
 [ 0.08691528]
 [ 0.01447127]]
[[ 0.05572128]
 [ 0.00937513]
 [ 0.02569385]
 [ 0.00355286]
 [ 0.36861022]
 [ 0.0416543 ]
 [ 0.64943736]
 [ 0.1038992 ]
 [ 0.02570519]
 [ 0.03198964]]
[[ 0.05476563]
 [ 0.2073698 ]
 [ 0.04423313]
 [ 0.02909225]
 [ 0.57545998]
 [ 0.47570293]
 [ 0

[[ 0.03126896]
 [ 0.00368794]
 [ 0.03194568]
 [ 0.00243081]
 [ 0.7801864 ]
 [ 0.05256502]
 [ 0.10901402]
 [ 0.12996738]
 [ 0.01518234]
 [ 0.00724737]]
[[ 0.02931043]
 [ 0.06509029]
 [ 0.01845242]
 [ 0.03984626]
 [ 0.07871246]
 [ 0.79725328]
 [ 0.05025839]
 [ 0.00993629]
 [ 0.1381515 ]
 [ 0.01421477]]
[[ 0.07288798]
 [ 0.01178138]
 [ 0.02462727]
 [ 0.01847047]
 [ 0.14743819]
 [ 0.02888562]
 [ 0.93202918]
 [ 0.00369116]
 [ 0.02577137]
 [ 0.02765731]]
[[  9.10971813e-04]
 [  9.32588683e-01]
 [  1.20413624e-02]
 [  1.55468892e-02]
 [  3.52583659e-02]
 [  5.75813998e-02]
 [  1.19850822e-02]
 [  3.35660131e-02]
 [  3.61001037e-02]
 [  2.92238063e-02]]
[[ 0.86254234]
 [ 0.00599328]
 [ 0.25134783]
 [ 0.04316204]
 [ 0.04658901]
 [ 0.07494237]
 [ 0.01107112]
 [ 0.03383024]
 [ 0.04217315]
 [ 0.00934211]]
[[ 0.93908915]
 [ 0.0025031 ]
 [ 0.02561668]
 [ 0.03076744]
 [ 0.04709168]
 [ 0.02853507]
 [ 0.0716649 ]
 [ 0.01360397]
 [ 0.02402426]
 [ 0.00840489]]
[[ 0.0041549 ]
 [ 0.84921617]
 [ 0.01709932]